In [1]:
function display_position(pos_matrix, time, stop)
    street = collect("-" ^ 100)
    street[stop] = '|'
    
    for c in 1:num_cars
        street[pos_matrix[c, time]] = Char('0' + c)
    end
    println(join(street))
end

display_position (generic function with 1 method)

In [2]:
num_cars = 5
lead_car = [-1, 1, 1]
cars = [lead_car]
for _ in 1:num_cars-1
    car = []
    speed = rand(1:3)
    spacing = rand(1:3)
    pos = cars[end][1] - 1 - spacing - rand(1:3)
    car = [pos, spacing, speed]
    push!(cars, car)
end

pos_of_last_car = cars[end][1]
shift_pos = 1 - pos_of_last_car

for car in cars
    car[1] += shift_pos
end

time = 100
pos_matrix = zeros(Int, num_cars, time)
for c in 1:num_cars
    pos_matrix[c, 1] = cars[c][1]
end

stop_distance = 90
println("t = 1")
display_position(pos_matrix, 1, stop_distance)
println()
for t in 2:time
    println("t = ", t)
    #println(cars)
    for i in 1:length(cars)
        car = cars[i]
        if i == 1
            if car[1] >= stop_distance          # lead car stops
                pos_matrix[i,t] = car[1]
                continue
            else
                car[1] += car[3]                # car moves according to its speed per timestep
            end
        else
            if (cars[i-1][1] - car[1] - car[3]) <= car[2] # if car is closer than designated spacing car should not move
                pos_matrix[i,t] = car[1]
                continue
            else
                car[1] += car[3]                # car moves according to its speed per timestep
            end
        end
        
        pos_matrix[i,t] = car[1]
    end
    
    #println(cars)
    display_position(pos_matrix, t, stop_distance)
    println()
end

t = 1
5-----4----3----2----1-------------------------------------------------------------------|----------

t = 2
--5------4---3--2-----1------------------------------------------------------------------|----------

t = 3
----5-------4--3---2---1-----------------------------------------------------------------|----------

t = 4
------5-----4--3---2----1----------------------------------------------------------------|----------

t = 5
--------5---4--3---2-----1---------------------------------------------------------------|----------

t = 6
--------5---4----3----2---1--------------------------------------------------------------|----------

t = 7
----------5----4---3--2----1-------------------------------------------------------------|----------

t = 8
----------5----4---3--2-----1------------------------------------------------------------|----------

t = 9
------------5-----4--3---2---1-----------------------------------------------------------|----------

t = 10
--------------5---4--